In [163]:
import subprocess
import os
import argparse
import pandas as pd
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging
from sqlalchemy import text
import os
import subprocess
import duckdb
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import datetime

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

In [138]:
datetime.datetime.now().year

2025

In [56]:
engine 
start_year=2024 
end_year=2024 
parquet_folder = 'data/raw/nflfastr/'

In [141]:
weekly_df = pd.read_sql("SELECT * FROM weekly_stats", engine)
players_df = pd.read_sql("SELECT * FROM players", engine)
teams_df = pd.read_sql('SELECT * FROM teams', engine)
games_df = pd.read_sql('SELECT * FROM games', engine)
depth_df = pd.read_sql('SELECT * FROM depth_chart', engine)
injuries_df = pd.read_sql('SELECT * FROM injuries', engine)
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)

In [ ]:
print('Fetching weather...')
API_KEY = '8332ALKSTAK2K47W3R6G9WK3L'

# Stadium → Location mapping
stadiums = {
    "Allegiant Stadium": "Las Vegas, NV",
    "AT&T Stadium": "Arlington, TX",
    "Bank of America Stadium": "Charlotte, NC",
    "Caesars Superdome": "New Orleans, LA",
    "FedExField": "Landover, MD",
    "Ford Field": "Detroit, MI",
    "Gillette Stadium": "Foxborough, MA",
    "Hard Rock Stadium": "Miami Gardens, FL",
    "Highmark Stadium": "Orchard Park, NY",
    "Lambeau Field": "Green Bay, WI",
    "Levi's Stadium": "Santa Clara, CA",
    "Lincoln Financial Field": "Philadelphia, PA",
    "Lucas Oil Stadium": "Indianapolis, IN",
    "Lumen Field": "Seattle, WA",
    "Mercedes-Benz Stadium": "Atlanta, GA",
    "MetLife Stadium": "East Rutherford, NJ",
    "M&T Bank Stadium": "Baltimore, MD",
    "NRG Stadium": "Houston, TX",
    "Paycor Stadium": "Cincinnati, OH",
    "Raymond James Stadium": "Tampa, FL",
    "SoFi Stadium": "Inglewood, CA",
    "Soldier Field": "Chicago, IL",
    "State Farm Stadium": "Glendale, AZ",
    "TIAA Bank Field": "Jacksonville, FL",
    "U.S. Bank Stadium": "Minneapolis, MN",
    "Arrowhead Stadium": "Kansas City, MO",
    "Acrisure Stadium": "Pittsburgh, PA",
    "Empower Field at Mile High": "Denver, CO",
    "FirstEnergy Stadium": "Cleveland, OH"  # Cleveland Browns
}

dome_stadiums = [
    "Lucas Oil Stadium",
    "Mercedes-Benz Stadium",
    "Caesars Superdome",
    "NRG Stadium",
    "Allegiant Stadium",
    "AT&T Stadium",
    "State Farm Stadium",
    "U.S. Bank Stadium",
    "Ford Field"
]

# Pull games from DB
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)
schedule_df['dome'] = schedule_df['stadium'].apply(lambda x: x in dome_stadiums)

# Fetch weather for each game
weather_records = []

BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

for _, game in schedule_df[schedule_df['dome'] == False].iterrows():
    stadium = game['stadium']
    location = stadiums.get(stadium)
    if not location:
        continue

    params = {
        "unitGroup": "us",
        "key": API_KEY,
        "include": "days",
        "elements": "datetime,temp,precip,windspeed",
        "contentType": "json"
    }

    response = requests.get(f"{BASE_URL}/{location}/{game['game_date']}", params=params)
    data = response.json()
    day = data["days"][0]

    weather_records.append({
        "season": game["season"],
        "week": game["week"],
        "stadium": stadium,
        "temperature": day.get("temp"),
        "precipitation": day.get("precip"),
        "wind_speed": day.get("windspeed"),
    })

# Ingest into DB
weather_df = pd.DataFrame(weather_records)
weather_df = weather_df.drop_duplicates().reset_index(drop=True)

Fetching weather...


In [192]:
weather_df

,season,week,stadium,temperature,precipitation,wind_speed
0,2022,1,SoFi Stadium,85.8,0.087,23.5
1,2022,1,Soldier Field,67.7,1.386,18.4
2,2022,1,Paycor Stadium,71.7,0.151,11.0
3,2022,1,Hard Rock Stadium,84.2,0.266,12.8
4,2022,1,MetLife Stadium,70.2,0.178,6.6


In [202]:
print('Fetching weather...')
API_KEY = '8332ALKSTAK2K47W3R6G9WK3L'

# Stadium → Location mapping
stadiums = {
    "Allegiant Stadium": "Las Vegas, NV",
    "AT&T Stadium": "Arlington, TX",
    "Bank of America Stadium": "Charlotte, NC",
    "Caesars Superdome": "New Orleans, LA",
    "FedExField": "Landover, MD",
    "Ford Field": "Detroit, MI",
    "Gillette Stadium": "Foxborough, MA",
    "Hard Rock Stadium": "Miami Gardens, FL",
    "Highmark Stadium": "Orchard Park, NY",
    "Lambeau Field": "Green Bay, WI",
    "Levi's Stadium": "Santa Clara, CA",
    "Lincoln Financial Field": "Philadelphia, PA",
    "Lucas Oil Stadium": "Indianapolis, IN",
    "Lumen Field": "Seattle, WA",
    "Mercedes-Benz Stadium": "Atlanta, GA",
    "MetLife Stadium": "East Rutherford, NJ",
    "M&T Bank Stadium": "Baltimore, MD",
    "NRG Stadium": "Houston, TX",
    "Paycor Stadium": "Cincinnati, OH",
    "Raymond James Stadium": "Tampa, FL",
    "SoFi Stadium": "Inglewood, CA",
    "Soldier Field": "Chicago, IL",
    "State Farm Stadium": "Glendale, AZ",
    "TIAA Bank Field": "Jacksonville, FL",
    "U.S. Bank Stadium": "Minneapolis, MN",
    "Arrowhead Stadium": "Kansas City, MO",
    "Acrisure Stadium": "Pittsburgh, PA",
    "Empower Field at Mile High": "Denver, CO",
    "FirstEnergy Stadium": "Cleveland, OH"  # Cleveland Browns
}

dome_stadiums = {
    "Lucas Oil Stadium",
    "Mercedes-Benz Stadium",
    "Caesars Superdome",
    "NRG Stadium",
    "Allegiant Stadium",
    "AT&T Stadium",
    "State Farm Stadium",
    "U.S. Bank Stadium",
    "Ford Field"
}

# Pull games from DB
schedule_df = pd.read_sql("SELECT season, week, stadium, game_date FROM games", engine)
schedule_df['dome'] = schedule_df['stadium'].apply(lambda x: x in dome_stadiums)

Fetching weather...


In [190]:
schedule_df

,season,week,stadium,game_date,dome
0,2022,1,SoFi Stadium,2022-09-09,False
1,2022,1,Mercedes-Benz Stadium,2022-09-11,True
2,2022,1,Soldier Field,2022-09-11,False
3,2022,1,Paycor Stadium,2022-09-11,False
4,2022,1,Ford Field,2022-09-11,True
...,...,...,...,...,...
1625,2025,18,Acrisure Stadium,2026-01-04,False
1626,2025,18,Levi's Stadium,2026-01-04,False
1627,2025,18,Raymond James Stadium,2026-01-04,False
1628,2025,18,EverBank Stadium,2026-01-04,False


In [209]:
schedule_df.drop(columns = ['dome'], inplace = True)

In [210]:
schedule_df

,season,week,stadium,game_date
0,2022,1,SoFi Stadium,2022-09-09
1,2022,1,Mercedes-Benz Stadium,2022-09-11
2,2022,1,Soldier Field,2022-09-11
3,2022,1,Paycor Stadium,2022-09-11
4,2022,1,Ford Field,2022-09-11
...,...,...,...,...
1625,2025,18,Acrisure Stadium,2026-01-04
1626,2025,18,Levi's Stadium,2026-01-04
1627,2025,18,Raymond James Stadium,2026-01-04
1628,2025,18,EverBank Stadium,2026-01-04


In [211]:
schedule_df['dome'] = schedule_df['stadium'].apply(lambda x: x in dome_stadiums)

In [212]:
schedule_df

,season,week,stadium,game_date,dome
0,2022,1,SoFi Stadium,2022-09-09,False
1,2022,1,Mercedes-Benz Stadium,2022-09-11,True
2,2022,1,Soldier Field,2022-09-11,False
3,2022,1,Paycor Stadium,2022-09-11,False
4,2022,1,Ford Field,2022-09-11,True
...,...,...,...,...,...
1625,2025,18,Acrisure Stadium,2026-01-04,False
1626,2025,18,Levi's Stadium,2026-01-04,False
1627,2025,18,Raymond James Stadium,2026-01-04,False
1628,2025,18,EverBank Stadium,2026-01-04,False


In [213]:
schedule_df['dome'].dtypes

dtype('bool')

In [214]:
schedule_df['dome'][0]

np.False_